In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [2]:
veri_dizini = "Data_2"

# Sınıfları sırala
siniflar = sorted([
    d for d in os.listdir(veri_dizini)
    if os.path.isdir(os.path.join(veri_dizini, d))
])


In [3]:
print("Sınıflar:", siniflar)

Sınıflar: ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']


In [4]:
goruntuler = []
etiketler = []

# Görüntü alımı diğer kodlarla tamamen aynı
for etiket, sinif_adi in enumerate(siniflar):
    klasor_yolu = os.path.join(veri_dizini, sinif_adi)
    dosyalar = os.listdir(klasor_yolu)
    
    for dosya in dosyalar:
        dosya_yolu = os.path.join(klasor_yolu, dosya)
        img = cv2.imread(dosya_yolu)
        if img is None:
            continue
            #Grayscale
        gri = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        yeniden_boyutlu = cv2.resize(gri, (64, 64))
        normalize_edilmis = yeniden_boyutlu / 255.0
        goruntuler.append(normalize_edilmis)
        etiketler.append(etiket)

goruntuler = np.array(goruntuler)
etiketler = np.array(etiketler)

In [5]:
print("Toplam görüntü sayısı:", len(goruntuler))

Toplam görüntü sayısı: 17460


In [6]:
import numpy as np
import cv2

def gabor_filtreleri_olustur():
    filtreler = []
    filtre_boyutu = 31  # Gabor çekirdeği boyutu

    for aci in np.arange(0, np.pi, np.pi / 4):  # 0°, 45°, 90°, 135°
        for sigma in (1, 3):  # düşük ve yüksek sigma değerleri
            for lamda in np.arange(np.pi / 4, np.pi, np.pi / 4):  # dalga boyları
                cekirdek = cv2.getGaborKernel(
                    (filtre_boyutu, filtre_boyutu),
                    sigma,
                    aci,
                    lamda,
                    gamma=0.5,
                    psi=0
                )
                filtreler.append(cekirdek)
    return filtreler

def gabor_ozellik_cikar(goruntuler, filtreler):
    ozellikler = []

    for img in goruntuler:
        img_ozellik = []
        for filtre in filtreler:
            uygulanmis = cv2.filter2D(img, cv2.CV_8UC3, filtre)
            ortalama = uygulanmis.mean()
            std = uygulanmis.std()
            img_ozellik.extend([ortalama, std])
        ozellikler.append(img_ozellik)

    return np.array(ozellikler)


In [7]:
gabor_filtreler = gabor_filtreleri_olustur()
gabor_ozellikleri = gabor_ozellik_cikar(goruntuler, gabor_filtreler)


In [8]:
# Gabor öznitelikleri ile eğitim ve test ayrımı
X_train, X_test, y_train, y_test = train_test_split(
    gabor_ozellikleri, etiketler,
    test_size=0.2,
    random_state=42,
    stratify=etiketler
)

print("Eğitim seti boyutu:", X_train.shape)
print("Test seti boyutu:", X_test.shape)


Eğitim seti boyutu: (13968, 48)
Test seti boyutu: (3492, 48)


In [9]:
#Diğer train dosylarındakiyle aynı sırada ilerleyeceğim SVM ,MLP,Random Forest ardından da XGBoost

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import time

# Denenecek SVM parametre kombinasyonları
svm_modeller = [
    {"C": 0.1, "gamma": "scale"},
    {"C": 1, "gamma": "scale"},
    {"C": 10, "gamma": "scale"},
    {"C": 1, "gamma": 0.1},
    {"C": 1, "gamma": 0.01},
    {"C": 1, "gamma": 0.001},
    {"C": 10, "gamma": 0.1},
    {"C": 10, "gamma": 0.01},
    {"C": 100, "gamma": 0.01},
    {"C": 100, "gamma": 0.001},
]

# Her model için eğitim ve test
for idx, parametreler in enumerate(svm_modeller, 1):
    print(f"\n--- Gabor + SVM Model {idx}: C={parametreler['C']}, gamma={parametreler['gamma']} ---")

    # Eğitim süresi başlangıcı
    egitim_basla = time.time()

    svm_model = SVC(
        C=parametreler["C"],
        gamma=parametreler["gamma"],
        kernel="rbf"
    )
    svm_model.fit(X_train, y_train)

    egitim_bitis = time.time()

    # Test süresi başlangıcı
    test_basla = time.time()
    y_tahmin = svm_model.predict(X_test)
    test_bitis = time.time()

    # Doğruluk ve rapor
    dogruluk = accuracy_score(y_test, y_tahmin)
    print("Doğruluk:", dogruluk)
    print(classification_report(y_test, y_tahmin, zero_division=0))

    # Süre bilgisi
    print(f"Eğitim süresi: {(egitim_bitis - egitim_basla):.2f} saniye")
    print(f"Test süresi: {(test_bitis - test_basla):.2f} saniye")



--- Gabor + SVM Model 1: C=0.1, gamma=scale ---
Doğruluk: 0.2992554410080183
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       563
           1       0.30      1.00      0.46      1045
           2       0.00      0.00      0.00       728
           3       0.00      0.00      0.00       707
           4       0.00      0.00      0.00       449

    accuracy                           0.30      3492
   macro avg       0.06      0.20      0.09      3492
weighted avg       0.09      0.30      0.14      3492

Eğitim süresi: 5.94 saniye
Test süresi: 3.36 saniye

--- Gabor + SVM Model 2: C=1, gamma=scale ---
Doğruluk: 0.30441008018327603
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       563
           1       0.30      0.98      0.46      1045
           2       0.29      0.02      0.05       728
           3       0.46      0.03      0.06       707
           4       0.00      0.00

In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
import time

# Denenecek MLP parametreleri
mlp_modeller = [
    {"hidden_layer_sizes": (100,), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (100, 50), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (200,), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (100,), "learning_rate_init": 0.01},
    {"hidden_layer_sizes": (100, 50, 25), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (150,), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (256, 128), "learning_rate_init": 0.0005},
    {"hidden_layer_sizes": (128, 64, 32), "learning_rate_init": 0.001},
    {"hidden_layer_sizes": (64, 64, 64), "learning_rate_init": 0.005},
    {"hidden_layer_sizes": (512, 256, 128, 64), "learning_rate_init": 0.001},
]

# Her model için döngü
for idx, parametreler in enumerate(mlp_modeller, 1):
    print(f"\n--- Gabor + MLP Model {idx}: Katmanlar={parametreler['hidden_layer_sizes']}, Öğrenme oranı={parametreler['learning_rate_init']} ---")

    # Eğitim süresi başlangıcı
    egitim_basla = time.time()
    
    mlp_model = MLPClassifier(
        hidden_layer_sizes=parametreler["hidden_layer_sizes"],
        learning_rate_init=parametreler["learning_rate_init"],
        max_iter=1000,
        early_stopping=True,
        random_state=42
    )
    mlp_model.fit(X_train, y_train)
    
    egitim_bitis = time.time()
    
    # Test süresi başlangıcı
    test_basla = time.time()
    y_tahmin = mlp_model.predict(X_test)
    test_bitis = time.time()
    
    # Doğruluk ve sınıflandırma raporu
    dogruluk = accuracy_score(y_test, y_tahmin)
    print("Doğruluk:", dogruluk)
    print(classification_report(y_test, y_tahmin, zero_division=0))
    
    # Süre bilgileri
    egitim_sure = egitim_bitis - egitim_basla
    test_sure = test_bitis - test_basla
    print(f"Eğitim süresi: {egitim_sure:.2f} saniye")
    print(f"Test süresi: {test_sure:.2f} saniye")



--- Gabor + MLP Model 1: Katmanlar=(100,), Öğrenme oranı=0.001 ---
Doğruluk: 0.34936998854524626
              precision    recall  f1-score   support

           0       0.45      0.02      0.03       563
           1       0.34      0.81      0.48      1045
           2       0.32      0.17      0.22       728
           3       0.39      0.13      0.20       707
           4       0.40      0.33      0.36       449

    accuracy                           0.35      3492
   macro avg       0.38      0.29      0.26      3492
weighted avg       0.37      0.35      0.28      3492

Eğitim süresi: 0.86 saniye
Test süresi: 0.00 saniye

--- Gabor + MLP Model 2: Katmanlar=(100, 50), Öğrenme oranı=0.001 ---
Doğruluk: 0.35223367697594504
              precision    recall  f1-score   support

           0       0.29      0.03      0.05       563
           1       0.34      0.80      0.48      1045
           2       0.34      0.18      0.23       728
           3       0.39      0.17      0.23

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import time

# Denenecek Random Forest parametreleri
rf_modeller = [
    {"n_estimators": 100, "max_depth": None},
    {"n_estimators": 100, "max_depth": 10},
    {"n_estimators": 200, "max_depth": None},
    {"n_estimators": 200, "max_depth": 20},
    {"n_estimators": 300, "max_depth": 15},
    {"n_estimators": 500, "max_depth": None},
]

# Her model için döngü
for idx, parametreler in enumerate(rf_modeller, 1):
    print(f"\n--- Gabor + Random Forest Model {idx}: n_estimators={parametreler['n_estimators']}, max_depth={parametreler['max_depth']} ---")
    
    # Eğitim süresi başlangıcı
    egitim_basla = time.time()
    
    rf_model = RandomForestClassifier(
        n_estimators=parametreler["n_estimators"],
        max_depth=parametreler["max_depth"],
        random_state=42
    )
    rf_model.fit(X_train, y_train)
    
    egitim_bitis = time.time()
    
    # Test süresi başlangıcı
    test_basla = time.time()
    y_tahmin = rf_model.predict(X_test)
    test_bitis = time.time()
    
    # Doğruluk ve sınıflandırma raporu
    dogruluk = accuracy_score(y_test, y_tahmin)
    print("Doğruluk:", dogruluk)
    print(classification_report(y_test, y_tahmin, zero_division=0))
    
    # Süre bilgileri
    egitim_sure = egitim_bitis - egitim_basla
    test_sure = test_bitis - test_basla
    print(f"Eğitim süresi: {egitim_sure:.2f} saniye")
    print(f"Test süresi: {test_sure:.2f} saniye")



--- Gabor + Random Forest Model 1: n_estimators=100, max_depth=None ---
Doğruluk: 0.39060710194730813
              precision    recall  f1-score   support

           0       0.38      0.15      0.21       563
           1       0.38      0.69      0.49      1045
           2       0.35      0.24      0.29       728
           3       0.36      0.26      0.30       707
           4       0.53      0.46      0.49       449

    accuracy                           0.39      3492
   macro avg       0.40      0.36      0.36      3492
weighted avg       0.39      0.39      0.37      3492

Eğitim süresi: 4.14 saniye
Test süresi: 0.05 saniye

--- Gabor + Random Forest Model 2: n_estimators=100, max_depth=10 ---
Doğruluk: 0.3559564719358534
              precision    recall  f1-score   support

           0       0.48      0.06      0.10       563
           1       0.34      0.78      0.48      1045
           2       0.35      0.15      0.21       728
           3       0.36      0.19      

In [13]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import time

# Denenecek XGBoost parametreleri
xgb_modeller = [
    {"max_depth": 3, "learning_rate": 0.1, "n_estimators": 100},
    {"max_depth": 5, "learning_rate": 0.1, "n_estimators": 200},
    {"max_depth": 7, "learning_rate": 0.05, "n_estimators": 200},
    {"max_depth": 5, "learning_rate": 0.2, "n_estimators": 300},
    {"max_depth": 4, "learning_rate": 0.01, "n_estimators": 500},
    {"max_depth": 6, "learning_rate": 0.1, "n_estimators": 100},
]

# Her model için eğitim ve test süreci
for idx, parametreler in enumerate(xgb_modeller, 1):
    print(f"\n--- Gabor + XGBoost Model {idx}: max_depth={parametreler['max_depth']}, learning_rate={parametreler['learning_rate']}, n_estimators={parametreler['n_estimators']} ---")
    
    # Eğitim süresi başlangıcı
    egitim_basla = time.time()
    
    xgb_model = XGBClassifier(
        max_depth=parametreler["max_depth"],
        learning_rate=parametreler["learning_rate"],
        n_estimators=parametreler["n_estimators"],
        eval_metric='mlogloss',
        random_state=42
    )
    xgb_model.fit(X_train, y_train)
    
    # Eğitim süresi bitişi
    egitim_bitis = time.time()
    
    # Test süresi başlangıcı
    test_basla = time.time()
    y_tahmin = xgb_model.predict(X_test)
    test_bitis = time.time()
    
    # Doğruluk ve raporlama
    dogruluk = accuracy_score(y_test, y_tahmin)
    print("Doğruluk:", dogruluk)
    print(classification_report(y_test, y_tahmin, zero_division=0))
    
    # Süre bilgileri
    egitim_sure = egitim_bitis - egitim_basla
    test_sure = test_bitis - test_basla
    print(f"Eğitim süresi: {egitim_sure:.2f} saniye")
    print(f"Test süresi: {test_sure:.2f} saniye")



--- Gabor + XGBoost Model 1: max_depth=3, learning_rate=0.1, n_estimators=100 ---
Doğruluk: 0.3467926689576174
              precision    recall  f1-score   support

           0       0.33      0.04      0.07       563
           1       0.35      0.74      0.47      1045
           2       0.31      0.17      0.22       728
           3       0.31      0.20      0.25       707
           4       0.42      0.34      0.38       449

    accuracy                           0.35      3492
   macro avg       0.35      0.30      0.28      3492
weighted avg       0.34      0.35      0.30      3492

Eğitim süresi: 0.61 saniye
Test süresi: 0.00 saniye

--- Gabor + XGBoost Model 2: max_depth=5, learning_rate=0.1, n_estimators=200 ---
Doğruluk: 0.3788659793814433
              precision    recall  f1-score   support

           0       0.31      0.11      0.16       563
           1       0.38      0.68      0.49      1045
           2       0.35      0.25      0.29       728
           3      